In [1]:
import config
import cx_Oracle
import re
import pandas as pd
import csv

pattern = re.compile(r'\s+')

In [6]:
# connect to Oracle DB
connection = None
try:
    connection = cx_Oracle.connect(
        config.username,
        config.password,
        config.dsn,
        encoding = "UTF-8"
    )
    print("Connected")
    print(connection.version)
except cx_Oracle.Error as error:
    print(error)

cursor = connection.cursor()

Connected
12.2.0.1.0


In [7]:
# set up style view and get the geometrytype of that style's feature
view_name = "V_HV_TRF_SYM_1"
feature_name = view_name[2:]
print(feature_name)
sql = "Select g3e_geometrytype from G3E_COMPONENT where G3E_NAME = '{name}'".format(name = feature_name)
cursor.execute(sql)
geometrytype = cursor.fetchall()[0][0]
print(geometrytype)

HV_TRF_SYM_1
OrientedPointGeometry


In [8]:
# hardcode condition to know which styletable for the given style view
if geometrytype == "PolygonGeometry":
    styletable = 'G3E_AREASTYLE'
elif geometrytype == "PolylineGeometry":
    styletable = 'G3E_LINESTYLE'
elif geometrytype == "OrientedPointGeometry":
    styletable = 'G3E_POINTSTYLE'
elif geometrytype == "TextPointGeometry":
    styletable = 'G3E_TEXTSTYLE'
elif geometrytype == "CompositePolylineGeometry":
    styletable = 'G3E_COMPOSITELINESTYLE'

print(styletable)

G3E_POINTSTYLE


In [9]:
# extract text that create the given style view
sql = "SELECT TEXT FROM USER_VIEWS WHERE VIEW_NAME = '{view_name}'".format(view_name = view_name)
cursor.execute(sql)
fetch = cursor.fetchall()
txts = fetch[0][0]
txts = txts.rpartition("CASE")[2]
txts = txts.rpartition("ELSE")[0]
txts = re.sub(pattern,'',txts)
txts = txts.split("WHEN")

print(txts)

['', "TRIM(D.CODETEXT)IN('PPA','PPM','PPR','PPX')ANDTRIM(E.CODETEXT)='RECTIFIER'THEN14800010", "TRIM(D.CODETEXT)IN('PPA','PPM','PPR','PPX')THEN14800020", "TRIM(D.CODETEXT)IN('ABA','ABM','ABR','ABX')ANDTRIM(E.CODETEXT)='RECTIFIER'THEN14800030", "TRIM(D.CODETEXT)IN('ABA','ABM','ABR','ABX')THEN14800040", "TRIM(D.CODETEXT)='INS'ANDTRIM(E.CODETEXT)='RECTIFIER'THEN14800050", "TRIM(D.CODETEXT)='INS'THEN14800060", "TRIM(D.CODETEXT)IN('RES','ABN')ANDTRIM(E.CODETEXT)='RECTIFIER'THEN14800070", "TRIM(D.CODETEXT)IN('RES','ABN')THEN14800080", "TRIM(D.CODETEXT)='APA'ANDTRIM(E.CODETEXT)='RECTIFIER'THEN14800090", "TRIM(D.CODETEXT)='APA'THEN14800100"]


In [11]:
# extract cause(logic condition) and effect(styleid) into dict
logics = {}

spl_chr = "THEN"
for txt in txts:
    if txt != "":
        logics.update({txt.rpartition(spl_chr)[2] : txt.rpartition(spl_chr)[0]})
    
print(logics)

{'14800010': "TRIM(D.CODETEXT)IN('PPA','PPM','PPR','PPX')ANDTRIM(E.CODETEXT)='RECTIFIER'", '14800020': "TRIM(D.CODETEXT)IN('PPA','PPM','PPR','PPX')", '14800030': "TRIM(D.CODETEXT)IN('ABA','ABM','ABR','ABX')ANDTRIM(E.CODETEXT)='RECTIFIER'", '14800040': "TRIM(D.CODETEXT)IN('ABA','ABM','ABR','ABX')", '14800050': "TRIM(D.CODETEXT)='INS'ANDTRIM(E.CODETEXT)='RECTIFIER'", '14800060': "TRIM(D.CODETEXT)='INS'", '14800070': "TRIM(D.CODETEXT)IN('RES','ABN')ANDTRIM(E.CODETEXT)='RECTIFIER'", '14800080': "TRIM(D.CODETEXT)IN('RES','ABN')", '14800090': "TRIM(D.CODETEXT)='APA'ANDTRIM(E.CODETEXT)='RECTIFIER'", '14800100': "TRIM(D.CODETEXT)='APA'"}


In [12]:
# get the relevant information for the given style id
styles = {}
for styleid in logics.keys():
    sqlstyle = "SELECT * FROM {table} WHERE g3e_sno = '{styleid}'".format(table = styletable, styleid = styleid)
    df=pd.read_sql(sqlstyle,con=connection)
    data = df.to_dict(orient = 'list')
    data.pop('G3E_SNO', None)
    data.pop('G3E_USERNAME', None)
    data.pop('G3E_EDITDATE', None)
    styles.update({styleid:data})

In [13]:
print(styles)
print(logics)

{'14800010': {'G3E_COLOR': [47615], 'G3E_FONTNAME': ['HECGIS_104'], 'G3E_SYMBOL': [']'], 'G3E_MASKSYMBOL': [None], 'G3E_SIZE': [3.98125065], 'G3E_USEMASK': [0], 'G3E_ALIGNMENT': [2], 'G3E_PDNO': [None], 'G3E_PLOTREDLINE': [1], 'G3E_STYLEUNITS': [1], 'G3E_ROTATION': [0], 'G3E_TRANSLUCENCY': [None], 'G3E_MASKTRANSLUCENCY': [None]}, '14800020': {'G3E_COLOR': [47615], 'G3E_FONTNAME': ['HECGIS_104'], 'G3E_SYMBOL': ['O'], 'G3E_MASKSYMBOL': [None], 'G3E_SIZE': [3.98125065], 'G3E_USEMASK': [0], 'G3E_ALIGNMENT': [2], 'G3E_PDNO': [None], 'G3E_PLOTREDLINE': [1], 'G3E_STYLEUNITS': [1], 'G3E_ROTATION': [0], 'G3E_TRANSLUCENCY': [None], 'G3E_MASKTRANSLUCENCY': [None]}, '14800030': {'G3E_COLOR': [11572735], 'G3E_FONTNAME': ['HECGIS_104'], 'G3E_SYMBOL': [']'], 'G3E_MASKSYMBOL': [None], 'G3E_SIZE': [3.98125065], 'G3E_USEMASK': [0], 'G3E_ALIGNMENT': [2], 'G3E_PDNO': [None], 'G3E_PLOTREDLINE': [1], 'G3E_STYLEUNITS': [1], 'G3E_ROTATION': [0], 'G3E_TRANSLUCENCY': [None], 'G3E_MASKTRANSLUCENCY': [None]}, '14

In [14]:
printdicts = {}
for styleid in logics.keys():
    printdicts.update({styleid: styles[styleid]})
    printdicts[styleid].update({"logic": logics[styleid]})
print(printdicts)

{'14800010': {'G3E_COLOR': [47615], 'G3E_FONTNAME': ['HECGIS_104'], 'G3E_SYMBOL': [']'], 'G3E_MASKSYMBOL': [None], 'G3E_SIZE': [3.98125065], 'G3E_USEMASK': [0], 'G3E_ALIGNMENT': [2], 'G3E_PDNO': [None], 'G3E_PLOTREDLINE': [1], 'G3E_STYLEUNITS': [1], 'G3E_ROTATION': [0], 'G3E_TRANSLUCENCY': [None], 'G3E_MASKTRANSLUCENCY': [None], 'logic': "TRIM(D.CODETEXT)IN('PPA','PPM','PPR','PPX')ANDTRIM(E.CODETEXT)='RECTIFIER'"}, '14800020': {'G3E_COLOR': [47615], 'G3E_FONTNAME': ['HECGIS_104'], 'G3E_SYMBOL': ['O'], 'G3E_MASKSYMBOL': [None], 'G3E_SIZE': [3.98125065], 'G3E_USEMASK': [0], 'G3E_ALIGNMENT': [2], 'G3E_PDNO': [None], 'G3E_PLOTREDLINE': [1], 'G3E_STYLEUNITS': [1], 'G3E_ROTATION': [0], 'G3E_TRANSLUCENCY': [None], 'G3E_MASKTRANSLUCENCY': [None], 'logic': "TRIM(D.CODETEXT)IN('PPA','PPM','PPR','PPX')"}, '14800030': {'G3E_COLOR': [11572735], 'G3E_FONTNAME': ['HECGIS_104'], 'G3E_SYMBOL': [']'], 'G3E_MASKSYMBOL': [None], 'G3E_SIZE': [3.98125065], 'G3E_USEMASK': [0], 'G3E_ALIGNMENT': [2], 'G3E_PDNO

In [15]:
df = pd.DataFrame.from_dict({(i): printdicts[i]
                                for i in printdicts.keys()},
                                orient = 'index')
print(df)

In [16]:
filename = view_name + ".csv"
csvdata = df.to_csv()
with open(filename,'w') as f:
    f.write(csvdata)
    

In [25]:
# close connection
if connection:
    cursor.close()
    connection.close()
    print("Connection closed")

Connection closed
